In [1]:
import csv
import json
import copy

In [2]:
root = 'dataset'
train_json = "../dataset/train.json"
json_file = None
with open(train_json, 'r') as f:
    json_file = json.load(f)

In [3]:
print(json_file.keys())
image_id = len(json_file["images"])
ann_id = len(json_file["annotations"])

# images_form = json_file["images"][0]
# ann_form = json_file["annotations"][0]
images_form = copy.deepcopy(json_file["images"][0])
ann_form = copy.deepcopy(json_file["annotations"][0])

print(image_id, ann_id)
print(images_form)
print(ann_form)

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
4883 23144
{'width': 1024, 'height': 1024, 'file_name': 'train/0000.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-26 14:44:23', 'id': 0}
{'image_id': 0, 'category_id': 0, 'area': 257301.66, 'bbox': [197.6, 193.7, 547.8, 469.7], 'iscrowd': 0, 'id': 0}


In [4]:
with open('pseudo.csv', 'r', newline='') as csv_file:
    reader = csv.reader(csv_file)
    header = True
    log = 1
    for line in reader:
        
        # header 부분 pass
        if header:
            header = False
            continue
        
        # labeling data, path load
        labeling, path = line
        
        # class, conf, bbox -> len:6
        labeling = labeling.split()
        if len(labeling) % 6 != 0:
            raise Exception("class, conf, bbox 형식 error")
        
        # images 추가
        i_form = copy.deepcopy(images_form)
        i_form["file_name"] = path
        i_form["id"] = image_id
        json_file["images"].append(i_form)
        if log == 1:
            print(i_form)
        # annotaion 추가
        for i in range(0, len(labeling), 6):
            category = int(labeling[i])
            conf = labeling[i+1]
            bbox = list(map(float, labeling[i+2:i+6]))
            area = bbox[2] * bbox[3]
            a_form = copy.deepcopy(ann_form)
            a_form["image_id"] = image_id
            a_form["category_id"] = category
            a_form["area"] = area
            a_form["bbox"] = bbox
            a_form["id"] = ann_id
            json_file["annotations"].append(a_form)
            ann_id += 1
            
            if log == 1:
                print(a_form)    
        image_id += 1
        
        log += 1

{'width': 1024, 'height': 1024, 'file_name': 'test/0000.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-26 14:44:23', 'id': 4883}
{'image_id': 4883, 'category_id': 7, 'area': 977207.3241459802, 'bbox': [602.4215087890625, 518.9984130859375, 955.3800048828125, 1022.8467407226562], 'iscrowd': 0, 'id': 23144}
{'image_id': 4883, 'category_id': 7, 'area': 215858.47876854986, 'bbox': [216.77316284179688, 49.2673225402832, 457.40618896484375, 471.9185791015625], 'iscrowd': 0, 'id': 23145}
{'image_id': 4883, 'category_id': 7, 'area': 337573.16370698065, 'bbox': [388.3249206542969, 188.35113525390625, 620.6612548828125, 543.8927612304688], 'iscrowd': 0, 'id': 23146}
{'image_id': 4883, 'category_id': 7, 'area': 564364.503858313, 'bbox': [448.34161376953125, 599.6484375, 645.4913940429688, 874.317626953125], 'iscrowd': 0, 'id': 23147}
{'image_id': 4883, 'category_id': 7, 'area': 237281.70723573864, 'bbox': [119.4267349243164, 417.924560546875, 328.9463500976562

In [5]:
with open("pseudo.json", 'w') as f:
    json.dump(json_file, f)